<span style='font-size:35px;'><span style='color:#860DFF;'>E</span><span style='color:#8311FF;'>r</span><span style='color:#8015FF;'>r</span><span style='color:#7D19FF;'>o</span><span style='color:#7B1DFF;'>r</span> <span style='color:#7525FF;'>4</span><span style='color:#7329FF;'>0</span><span style='color:#702EFF;'>4</span> <span style='color:#6B36FF;'>3</span><span style='color:#683AFF;'>7</span><span style='color:#653EFF;'>3</span><span style='color:#6342FF;'>5</span><span style='color:#6046FF;'>C</span> <span style='color:#5B4FFF;'>C</span><span style='color:#5853FF;'>o</span><span style='color:#5557FF;'>d</span><span style='color:#525BFF;'>e</span> <span style='color:#4D63FF;'>E</span><span style='color:#4A67FF;'>x</span><span style='color:#486BFF;'>p</span><span style='color:#4570FF;'>l</span><span style='color:#4274FF;'>a</span><span style='color:#4078FF;'>n</span><span style='color:#3D7CFF;'>a</span><span style='color:#3A80FF;'>t</span><span style='color:#3884FF;'>i</span><span style='color:#3588FF;'>o</span><span style='color:#328CFF;'>n</span></span>

<hr></hr>

<span style='font-size:27px;'><span style='color:#860DFF;'>O</span><span style='color:#7B1DFF;'>v</span><span style='color:#702EFF;'>e</span><span style='color:#653EFF;'>r</span><span style='color:#5B4FFF;'>v</span><span style='color:#505FFF;'>i</span><span style='color:#4570FF;'>e</span><span style='color:#3A80FF;'>w</span></span>

There are a few interesting bits of code I would like to explain. I'll divide them into three categories:
<ol>
    <li>X-Drive Logic,</li>
    <li>Shooting (flywheel, auto aiming), and</li>
    <li>Auton (wheel odometry)</li>
</ol>

First, a few miscellaneous points. Our school does not let us use VSCode or any third-party libraries, meaning we use the base V5 Pro code editor and have built all of our code from scratch. Also, we've saved all of our work and have it all documented through our GitHub commit history. Our code is open source and available for all to view.

<hr></hr>

<span style='font-size:27px;'><span style='color:#860DFF;'>X</span><span style='color:#7F17FF;'>-</span><span style='color:#7821FF;'>D</span><span style='color:#722BFF;'>r</span><span style='color:#6B35FF;'>i</span><span style='color:#643FFF;'>v</span><span style='color:#5E49FF;'>e</span> <span style='color:#515EFF;'>L</span><span style='color:#4A68FF;'>o</span><span style='color:#4372FF;'>g</span><span style='color:#3D7CFF;'>i</span><span style='color:#3686FF;'>c</span></span>

Our robot uses an x-drive drivetrain, which, though not incredibly common, already has pretty standardized code. This drivetrain can go forward, backwards, and strafe side-to-side. Our x-drive is unique because it is field oriented; it's the closest you can get to running a swerve drive on a vex bot. Field oriented driving means that, no matter what direction the robot is facing, it goes forward relative to you/the field, not itself. So if the robot is facing left, and you tell it to go forward, it'll strafe to its right.

<img src="https://raw.githubusercontent.com/elee012345/vex-spin-up/main/code-explanations/images/field%20oriented.png" alt="Field Oriented Driving Diagram">
(Outdated CAD btw)

Code is below:
```cpp
double headingRadians = Inertial2.heading() * 3.14159/180;
double yPos = con1.Axis3.position(pct);
double xPos = con1.Axis4.position(pct);
double sineHeading = sin(headingRadians);
double cosHeading = cos(headingRadians);
// rotate the controller x/y coordinates by the negative of the robot heading
// robot is facing 30 degrees and we tell it to go forward, then it needs to strafe -30 degrees

// this is some linear algebra
double rotatedYPos = xPos * sineHeading + yPos * cosHeading;
double rotatedXPos = xPos * cosHeading - yPos * sineHeading;
```

<hr></hr>

<span style='font-size:27px;'><span style='color:#860DFF;'>S</span><span style='color:#7B1DFF;'>h</span><span style='color:#702EFF;'>o</span><span style='color:#653EFF;'>o</span><span style='color:#5B4FFF;'>t</span><span style='color:#505FFF;'>i</span><span style='color:#4570FF;'>n</span><span style='color:#3A80FF;'>g</span></span>

Shooting I'll further divide into two categories. The first is the flywheel code, and the second is our auto-aiming code.

First I'll explain our flywheel code. Our robot has the ability to shoot from anywhere on the field, since, using the vision sensor, we're able to estimate how far our robot is from the goal and the adjust the flywheel speed accordingly. This is an advantage over catapults, which have a fixed place on the field they have to launch from.
We discovered that Vex's built in 'speed' methods don't actually directly control the motors. Instead, our speed input is first run through a PID controller that isn't tuned for our flywheel and gives an output that makes the flywheel velocity oscillate. Since the speed wasn't consistent, we weren't able to score shots consistently. We found that, to bypass this, we can instead use voltage, which ignores the preset, badly tuned PID controller entirely.

```cpp
// Vex internal PID means this code isn't turned correctly for our flywheel
vex::motor::spin(directionType::fwd, 100, velocityUnits::pct)

// Controlling the flywheel motors with voltage bypasses the built-in Vex PID
vex::motor::spin(directionType::fwd, 12, voltageUnits::volt)
```

Secondly, our robot auto-aims onto the goals. The code for this is really quite simple; all we have to do check the goal's offset from our vision sensor's center and run that through a PID controller. Then, our robot turns by that amount.
The auto aiming is more consistent with the red goal than the blue goal. I won't explain in detail why, but in brief, the blue goal is dark, and there are a lot of common colors (like black) that the vision sensor confused with the blue goal. At TSA, where everyone is wearing blue shirts, we will have to manually aim the robot onto the goal.

```cpp
// get middle of targetted object
// The vision sensor grabs the top left corner of the target, so we have to do a little bit
// of math to convert that to the target's center.
int targetMid = VisionSensor.largestObject.originX + (VisionSensor.largestObject.width / 2);
int error = screenCenter - targetMid;
// 'goal' is our PID controller
goal.setValues(0.2, 0.003, 0, targetMid);
int turning;

// i bound, only accumulates i if close enough to the goal
if ( error < 20 ) {
    turning = goal.getOutput(screenCenter, 0, true, targetMid);
} else {
    goal.resetError();
    turning = goal.getOutput(screenCenter, 0, false, targetMid);
}
```

<hr></hr>

<span style='font-size:27px;'><span style='color:#860DFF;'>A</span><span style='color:#8113FF;'>u</span><span style='color:#7C1AFF;'>t</span><span style='color:#7821FF;'>o</span><span style='color:#7328FF;'>n</span><span style='color:#6F2FFF;'>o</span><span style='color:#6A36FF;'>m</span><span style='color:#663DFF;'>o</span><span style='color:#6144FF;'>u</span><span style='color:#5D4BFF;'>s</span> <span style='color:#5459FF;'>O</span><span style='color:#4F60FF;'>d</span><span style='color:#4B67FF;'>o</span><span style='color:#466EFF;'>m</span><span style='color:#4275FF;'>e</span><span style='color:#3D7CFF;'>t</span><span style='color:#3983FF;'>r</span><span style='color:#348AFF;'>y</span></span>

Since we don't have distance sensors or a GPS sensor, our robot runs its autonomous routine based on wheel odometry. Odometry is just a fancy word for saying that we estimate the robot's position and movements based off of the degrees each wheel has turned. Combined with our inertial sensor, we've implemented easy-to-use methods that make our auton routine easy to program. 
For example, the goTo method takes three parameters: the y to drive, x to drive, and the time it should take. So, if I want the robot to stafe 5 inches to the right in 1 second, I just have to call:
 ```cpp 
 goTo(0, 5, 1);
 ```
 Similar methods have been implemented for turning. Most important is the turnToAbsolute method. If we want the robot to turn to a very precise heading, wheel odometry can be inconsistent because of a combination of gear and motor play. Instead, we can get much more consistent results with the inertial sensor, which tells us exactly where the robot is facing. The turnToAbsolute method turns to a specified angle from 0-359, regardless of where the robot is facing when the method is called (which would be turnToRelative). By doing some fancy math, the robot calculates the fastest way to turn to the specfied angle and goes there. We don't have to worry about robot inconsistency this way, since the inertial sensor always knows where it is and where it's facing.

Here's an example below:
```cpp
void autonR(void) {
    AutonCommands::goTo(0,16,0.7);
    AutonCommands::goTo(-2,0,0.1);
    AutonCommands::spinUpFlywheel(12);
    AutonCommands::starting();
    AutonCommands::doRollerfast();
    AutonCommands::goTo(2.5,0,0.3);
    AutonCommands::turnToAbsolute(330, 60);
    AutonCommands::turnToAbsolute(335, 20);
    AutonCommands::shoot1(1.5,13,2);
    AutonCommands::turnToAbsolute(190, 80);
    AutonCommands::turnToAbsolute(200, 20);
    AutonCommands::spinIntake();
    AutonCommands::goTo(-60,0,1);
    AutonCommands::spinUpFlywheel(10);
    AutonCommands::turnToAbsolute(295, 80);
    AutonCommands::shoot1(1.75,11,3);
    AutonCommands::stopIntake();
}
```
This is as close as text code can get to block code. I (the lead programmer of the team) missed a meeting and asked a mechanical member to help out with our auton. Generally, it's a good idea for the separate teams to keep with their roles. However, since these methods are so easy to implement, our mech member was able to successfully code part of our skills auton routine while I was gone. All of our code has comments and is all sufficiently documented to the point where anybody on the team can use it.

Wheel odometry isn't actually too difficult to calculate. We convert from wheel diameter and gear ratio to circumference to rotations based on how far the robot is supposed to drive, and do a few more calculations to determine the speed at which each wheel's motor should turn at per second (since we're strafing to x/y, the wheels are not necessarily turning at the same speed). The most difficult part about it is that an x-drive has wheels that are vectored outwards at 45 degrees. This means that to drive forward, we can't directly calculate how far a normal wheel would drive. By doing a little bit of trigonometry, we can solve for how much we need to scale the driving by, the square root of two. Without overcomplicating it too much, we use the special 45 45 90 triangle identity and solve for the hypotenuse.

Code is below:
```cpp
double WHEEL_DIAMETER = 4; // inches
double CIRCUMFERENCE = 3.14159 * WHEEL_DIAMETER;


// 84 teeth on wheels and 36 on motors
// also 7/3 gear ratio
// changed gear ratio back to 1:1
double GEAR_RATIO = 1/1;
double x_rotations = (xToGo / CIRCUMFERENCE) * GEAR_RATIO;
double y_rotations = (yToGo / CIRCUMFERENCE) * GEAR_RATIO;
/*
*     robot wheels looks like this:
*   /   \
*   \   /
*
*   they vector outward at 45 degrees each
*  however, we want to drive forward a certain amount of inches
* not driving 45 degrees in a direction
* if you extend everything out then you get a square
* with a line going from one corner to another that is as far as you want
* your robot to drive.
* You get a 45 45 90 triangle, where the legs are each x and the hypotenuse is x root 2
* that means each of the legs is the distance you want to go divided by root 2
* so we have each of the robot motors turn that far instead
*
* then we multiply by 1.1 to overshoot a little bit because our motors have play
*/
double xDegrees = x_rotations * 360 / sqrt(2) * 1.1;
double yDegrees = y_rotations * 360  / sqrt(2) * 1.1;

// different direction where the wheels are pointing so different degrees to turn
double front_left_degrees = xDegrees + yDegrees;
double front_right_degrees = xDegrees - yDegrees;
double back_left_degrees = xDegrees - yDegrees;
double back_right_degrees = xDegrees + yDegrees;

double front_left_degrees_per_second = front_left_degrees/secondsToComplete;
double front_right_degrees_per_second = front_right_degrees/secondsToComplete;
double back_left_degrees_per_second = back_left_degrees/secondsToComplete;
double back_right_degrees_per_second = back_right_degrees/secondsToComplete;
```